In [ ]:
# This is the project of Lukyanchuk Kirill and Akhmetov Nygmetzhan IT-2105 by the subject Advanced Programming
# Program is called "Apple Inc." stock price prediction using Facebook prophet and ready dataset from Nasdaq website

In [82]:
#import pandas library
import pandas as pd

In [98]:
#import our dataset from PC
from google.colab import files
files.upload()

Saving AAPL.csv to AAPL.csv


{'AAPL.csv': b'Date,Close/Last,Volume,Open,High,Low\r\n02/17/2023,$152.55,59144120,$152.35,$153,$150.85\r\n02/16/2023,$153.71,68167940,$153.51,$156.33,$153.3475\r\n02/15/2023,$155.33,65669250,$153.11,$155.5,$152.88\r\n02/14/2023,$153.2,61707570,$152.12,$153.77,$150.86\r\n02/13/2023,$153.85,62199010,$150.952,$154.26,$150.92\r\n02/10/2023,$151.01,57450710,$149.46,$151.3401,$149.22\r\n02/09/2023,$150.87,56007140,$153.775,$154.33,$150.42\r\n02/08/2023,$151.92,64120080,$153.88,$154.58,$151.168\r\n02/07/2023,$154.65,83322550,$150.64,$155.23,$150.64\r\n02/06/2023,$151.73,69858310,$152.575,$153.1,$150.78\r\n02/03/2023,$154.5,154357300,$148.03,$157.38,$147.83\r\n02/02/2023,$150.82,118339000,$148.9,$151.18,$148.17\r\n02/01/2023,$145.43,77663630,$143.97,$146.61,$141.32\r\n01/31/2023,$144.29,65874460,$142.7,$144.34,$142.28\r\n01/30/2023,$143,64015270,$144.955,$145.55,$142.85\r\n01/27/2023,$145.93,70555840,$143.155,$147.23,$143.08\r\n01/26/2023,$143.96,54105070,$143.17,$144.25,$141.9\r\n01/25/2023,

In [99]:
#create dataframe that copy all information from our dataset,  remove $ sign from the values of "Close/Last" column, because pandas reads only the numeric values,  and then invert our list of dataset
df = pd.read_csv('AAPL.csv')
df['Close/Last'] = df['Close/Last'].str.replace('$', '')

df['Close/Last'] = pd.to_numeric(df['Close/Last'])

df = df.iloc[::-1]
df

<ipython-input-99-29b34e8069f2>:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



,Date,Close/Last,Volume,Open,High,Low
1258,02/20/2018,42.9625,134762640,$43.0125,$43.565,$42.855
1257,02/21/2018,42.7675,149512280,$43.2075,$43.53,$42.7525
1256,02/22/2018,43.1250,123815040,$42.95,$43.4875,$42.9275
1255,02/23/2018,43.8750,135088200,$43.4175,$43.9125,$43.385
1254,02/26/2018,44.7425,149414680,$44.0875,$44.8475,$44.0525
...,...,...,...,...,...,...
4,02/13/2023,153.8500,62199010,$150.952,$154.26,$150.92
3,02/14/2023,153.2000,61707570,$152.12,$153.77,$150.86
2,02/15/2023,155.3300,65669250,$153.11,$155.5,$152.88
1,02/16/2023,153.7100,68167940,$153.51,$156.33,$153.3475


In [100]:
 #rename "Date" and "Close/Last" columns to "ds" and "y", because Facebook prophet reads and outputs in 'ds' and 'y' format 
df = df[['Date', 'Close/Last']]
df = df.rename(columns={'Date': 'ds', 'Close/Last': 'y'})
df

,ds,y
1258,02/20/2018,42.9625
1257,02/21/2018,42.7675
1256,02/22/2018,43.1250
1255,02/23/2018,43.8750
1254,02/26/2018,44.7425
...,...,...
4,02/13/2023,153.8500
3,02/14/2023,153.2000
2,02/15/2023,155.3300
1,02/16/2023,153.7100


In [101]:
#separate last 30 days and place them into the array named "last30days_for_pred" in order to compare predicted values with the actual values from this array
last30days_for_pred = df[len(df)-30:]
last30days_for_pred

,ds,y
29,01/06/2023,129.62
28,01/09/2023,130.15
27,01/10/2023,130.73
26,01/11/2023,133.49
25,01/12/2023,133.41
24,01/13/2023,134.76
23,01/17/2023,135.94
22,01/18/2023,135.21
21,01/19/2023,135.27
20,01/20/2023,137.87


In [102]:
#remove last 30 days from dataframe
df = df[:-30]
df

,ds,y
1258,02/20/2018,42.9625
1257,02/21/2018,42.7675
1256,02/22/2018,43.1250
1255,02/23/2018,43.8750
1254,02/26/2018,44.7425
...,...,...
34,12/29/2022,129.6100
33,12/30/2022,129.9300
32,01/03/2023,125.0700
31,01/04/2023,126.3600


In [31]:
# installation of pystan
pip install pystan==2.19.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: pystan
    Found existing installation: pystan 3.3.0
    Uninstalling pystan-3.3.0:
      Successfully uninstalled pystan-3.3.0


In [33]:
# installation of Facebook prophet
pip install fbprophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for fbprophet: filename=fbprophet-0.7.1-py3-none-any.whl size=9536306 sha256=a20c8f06f0b8972166aa7bea5c9efdf156d7426e50d860f7c1c59729f0af3fa6
  Stored in directory: /root/.cache/pip/wheels/d0/d2/ae/c579b7fd160999d35908f3cb8ebcad7ef64ecaca7b78e4c3c8
Successfully built fbprophet


In [107]:
# here prophet library imported, object of prophet, future object is created and predicted and model is trained
from fbprophet import Prophet
fbp = Prophet(daily_seasonality = True)
fbp.fit(df)
future = fbp.make_future_dataframe(periods=730)
forecast = fbp.predict(future)

In [108]:
# plot from fbrprophet is used to show the graph
from fbprophet.plot import plot_plotly
plot_plotly(fbp, forecast)

In [116]:
# here we can predict any value from 2018 till 2025
forecast[forecast.ds == '02/14/2023']['yhat']

1268    138.452488
Name: yhat, dtype: float64

In [106]:
# here we can output the actual value in order to compare and check the difference
last30days_for_pred[last30days_for_pred.ds == '02/14/2023']['y']

3    153.2
Name: y, dtype: float64